In [116]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

In [117]:
# Importing the dataset
from sklearn.model_selection import train_test_split
# 读取数据， 第一行是列名
labels = pd.read_csv('../data/labels.csv', header=0)
featuress = pd.read_csv('../data/features.csv', header=0)
# 将数据转换为numpy数组
# 如果label为yes，转换为1，否则转换为0
for i in range(len(labels)):
    if labels['yes/no'][i] == 'yes':
        labels['yes/no'][i] = 1
    else:
        labels['yes/no'][i] = 0
labels = np.array(labels).astype(int)
featuress = np.array(featuress).astype(float)
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(featuress, labels, test_size=0.3, random_state=0)

# 将X_train 增加一个维度，值为1
X_train = np.expand_dims(X_train, axis=1)
X_test = np.expand_dims(X_test, axis=1)

In [146]:
 # 导入所需库
import torch
import torch.nn as nn 
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn.functional as F 

# 定义超参数
input_dim = 1 # 输入数据维度 
num_classes = 2 # 分类数
batch_size = 64
learning_rate = 1e-3
epochs = 10

# 定义CNN模型
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv1d(1, 32, kernel_size=5), 
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size=1), 
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        
        self.fc2 = nn.Linear(64, num_classes) 
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out) 
        out = out.reshape(out.size(0), -1)
        out = self.fc2(out)
        return out

# 实例化模型,判断GPU是否可用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN().to(device) 
train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).long())
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练
for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
        x = x.to(device) # 数据到GPU
        y = y.to(device) 
        optimizer.zero_grad()
        #print(x.shape)
        outputs = model(x)   
        # 将y变成0D的tensor
        y = y.squeeze(1)
        outputs = F.log_softmax(outputs, dim=1)
        # print(outputs.shape)
        # print(y.shape)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        if (i+1) % 10 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, i+1, len(train_loader), loss.item()))
# 测试
model.eval()

# 计算测试集准确率
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x = x.to(device)
        y = y.to(device)
        outputs = model(x)
        y = y.squeeze(1)
        _, predicted = torch.max(outputs.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))


Epoch [1/10], Step [10/495], Loss: 7.7850
Epoch [1/10], Step [20/495], Loss: 6.5252
Epoch [1/10], Step [30/495], Loss: 2.6516
Epoch [1/10], Step [40/495], Loss: 2.0529
Epoch [1/10], Step [50/495], Loss: 0.9378
Epoch [1/10], Step [60/495], Loss: 68.2078
Epoch [1/10], Step [70/495], Loss: 8.5421
Epoch [1/10], Step [80/495], Loss: 9.4222
Epoch [1/10], Step [90/495], Loss: 1.3931
Epoch [1/10], Step [100/495], Loss: 17.7477
Epoch [1/10], Step [110/495], Loss: 1.9715
Epoch [1/10], Step [120/495], Loss: 1.4531
Epoch [1/10], Step [130/495], Loss: 1.3506
Epoch [1/10], Step [140/495], Loss: 2.4509
Epoch [1/10], Step [150/495], Loss: 1.7104
Epoch [1/10], Step [160/495], Loss: 4.6592
Epoch [1/10], Step [170/495], Loss: 13.2153
Epoch [1/10], Step [180/495], Loss: 0.9928
Epoch [1/10], Step [190/495], Loss: 1.1460
Epoch [1/10], Step [200/495], Loss: 131.2766
Epoch [1/10], Step [210/495], Loss: 25.7539
Epoch [1/10], Step [220/495], Loss: 88.3010
Epoch [1/10], Step [230/495], Loss: 17.3449
Epoch [1/10]

In [135]:
|


SyntaxError: invalid syntax (2953399504.py, line 1)